# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import seaborn as sb
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather = pd.read_csv("../output_data/cities.csv")
weather.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Hilo,90,US,1585696347,69,19.73,-155.09,24.44,2.60
1,1,Thompson,75,CA,1585695783,67,55.74,-97.86,-7.00,7.70
2,2,Mataura,12,NZ,1585696348,87,-46.19,168.86,15.56,0.45
3,3,Mount Gambier,40,AU,1585696349,100,-37.83,140.77,12.00,2.10
4,4,Whitehorse,75,CA,1585696349,31,60.72,-135.05,-7.22,6.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)


In [6]:
locations = weather[["Lat","Lng"]]
humidity = weather["Humidity"]


figure_layout = {
    "width": "800px",
    "height": "800px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}

fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, max_intensity=max(humidity))
fig.add_layer(heatmap_layer)
heatmap_layer.point_radius = 8
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#--- Temperature betwen 20 y 25 Grades
#--- humidity between 45 and 65

ideal_weather = weather.loc[(weather["Max Temp"]>15) & (weather["Max Temp"]<25)]

ideal_humidity = ideal_weather.loc[(ideal_weather["Humidity"] > 45)&(ideal_weather["Humidity"]< 65)]
ideal_wind = ideal_humidity.loc[ideal_weather["Wind Speed"]<2]
final_destinations = ideal_wind.dropna()
final_destinations

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
38,38,Castro,18,BR,1585696364,63,-24.79,-50.01,18.99,1.04
113,113,Lang Son,99,VN,1585696388,61,21.83,106.73,16.56,1.05
156,156,Te Anau,14,NZ,1585696407,51,-45.42,167.72,15.15,0.65
170,170,Isiolo,50,KE,1585696413,54,0.35,37.58,20.77,0.94
194,194,Airai,95,TL,1585696420,58,-8.93,125.41,20.44,0.87
378,378,Sekoma,99,BW,1585696521,51,-24.40,23.88,20.05,1.88
406,406,Sawākin,0,SD,1585696530,46,19.11,37.33,24.15,0.69
411,411,Moyale,2,KE,1585696532,60,3.52,39.06,21.16,1.63
426,426,Ahipara,100,NZ,1585696537,63,-35.17,173.17,20.11,1.46
509,509,Hokitika,36,NZ,1585696562,57,-42.72,170.97,16.51,1.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
trip = final_destinations.copy().reset_index()
trip["Hotel Name"] = ""
trip.drop(["Unnamed: 0","index"], axis=1,inplace=True)
trip

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Castro,18,BR,1585696364,63,-24.79,-50.01,18.99,1.04,
1,Lang Son,99,VN,1585696388,61,21.83,106.73,16.56,1.05,
2,Te Anau,14,NZ,1585696407,51,-45.42,167.72,15.15,0.65,
3,Isiolo,50,KE,1585696413,54,0.35,37.58,20.77,0.94,
4,Airai,95,TL,1585696420,58,-8.93,125.41,20.44,0.87,
5,Sekoma,99,BW,1585696521,51,-24.40,23.88,20.05,1.88,
6,Sawākin,0,SD,1585696530,46,19.11,37.33,24.15,0.69,
7,Moyale,2,KE,1585696532,60,3.52,39.06,21.16,1.63,
8,Ahipara,100,NZ,1585696537,63,-35.17,173.17,20.11,1.46,
9,Hokitika,36,NZ,1585696562,57,-42.72,170.97,16.51,1.55,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_radius = 5000
target_type = "lodging"


for index, row in trip.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    target_coordinates = f"{lat}, {lng}"
    params ={ 
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
        
    }
    
    try:
        response = requests.get(base_url,params=params)
        hotels = response.json()
        name = hotels["results"][0]["name"]
        trip.loc[index,"Hotel Name"]=name          
        
    except (IndexError, KeyError):
        pass


In [10]:
trip['Hotel Name'].replace('', np.nan, inplace=True)
trip.dropna(subset=["Hotel Name"],inplace=True)
trip

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Castro,18,BR,1585696364,63,-24.79,-50.01,18.99,1.04,CHACARA BAILLY
1,Lang Son,99,VN,1585696388,61,21.83,106.73,16.56,1.05,KHAINGAHOTEL
2,Te Anau,14,NZ,1585696407,51,-45.42,167.72,15.15,0.65,Kingsgate Hotel Te Anau
3,Isiolo,50,KE,1585696413,54,0.35,37.58,20.77,0.94,Gaddisa Hotel
4,Airai,95,TL,1585696420,58,-8.93,125.41,20.44,0.87,Juvinal Martin
6,Sawākin,0,SD,1585696530,46,19.11,37.33,24.15,0.69,فندق السادات
7,Moyale,2,KE,1585696532,60,3.52,39.06,21.16,1.63,BISMILLAHI HOTEL
8,Ahipara,100,NZ,1585696537,63,-35.17,173.17,20.11,1.46,Ahipara Holiday Park
9,Hokitika,36,NZ,1585696562,57,-42.72,170.97,16.51,1.55,Beachfront Hotel Hokitika
10,Marawi,96,PH,1585696576,58,8.00,124.29,24.36,1.03,Derogongan Residence


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in trip.iterrows()]
marker_locations = trip[["Lat", "Lng"]]


In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(marker_locations,info_box_content=hotel_info)
fig.add_layer(markers)


# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…